### Chris Sutton
### 
### Lab#2
#### 
#### As of Lab 2 this course is already getting quite interesting with classifing languages based on a trained LM. The key to to completing the Lab was understaning the LM data structure and being able to navigate and parse it to pull in the required probabilites for estimation. I completed the lab using split methods to sort data to my liking but one the last problem I moved over to regEx. I haven't ever had much experience with regEx but it's has come up on some coding entrace tests for the US government so I want to get a better intuition with it. 

In [1]:
from collections import *
from math import *
import pprint
import operator
from random import random

In [2]:
# charlm.py: exmaple code for lab 2 in 605.646

# This lab is inspired from a similar lab of Chris Callison-Burch's,
# which was in turn inspired by a blot post by Yoav Goldberg:
# https://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139
#
# We fixed the example code to pad each sentence/line with a start indicator instead
# of only padding the very first sentence.

# Convert counts to probabilities for successor chars in a given context
def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]

In [3]:
# Read a training file and produce a language model
def train_char_lm(fname, order):
    data = open(fname,encoding='utf-8').read()
    sents = data.split('\n')
    lm = defaultdict(Counter)
    for s in sents:
        pad = "~" * order
        data = pad + s + '\n'
#         print(data)
        for i in range(len(data)-order):
            history, char = data[i:i+order], data[i+order]
#             print(history, char)
            lm[history][char]+=1
#             print(lm)
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

In [4]:
# Given a character LM, randomly choose a next character given this history and return it
def generate_letter(lm, history, order):
        history = history[-order:]
        dist = lm[history]
        x = random()
        for c,v in dist:
            x = x - v
            if x <= 0: return c

In [5]:
# Generate a random text by repeatedly calling generate_letter
def generate_text(lm, order, nletters=1000):
    history = "~" * order
    out = []
    for i in range(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
        if c == '\n':
            history = "~" * order
    return "".join(out)

In [6]:
# Print alternatives given this context
def print_probs(lm, history):
    probs = sorted(lm[history],key=lambda x:(-x[1],x[0]))
    pp = pprint.PrettyPrinter()
    pp.pprint(probs)

In [7]:
# Compute the per-char perplexity of a text, using an input LM.  Returns infinity if a probability isn't found in the model
def perplexity(text, lm, order=4):
    # Pad the input with "~" chars.  This handles the case where order > len(text).
    pad = "~" * order
    data = pad + text
    print(data)
    result=[]
    flag =False
    for i in range(len(data)-order):
#         print(data[i:i+order])
#         print(lm[data[i:i+order]])
        if data[i:i+order] not in lm:
            flag = True
        else:
            subsetoflm =lm[data[i:i+order]]
        for key, value in subsetoflm:
            if key ==data[i+order]:
#                 print(key,value)
                result.append(log2(value))
#     pow(2,-1/len(text)*sum(result)))
#         print_probs(lm, data[i:i+order])
#         probs = sorted(lm[data[i:i+order]],key=lambda x:(-x[1],x[0]))
#     pp = pprint.PrettyPrinter()
#     pp.pprint(probs)
    # Loop over data string and find probs and use to compute perplexity
    if flag ==True:
        return inf
    else:
        return  pow(2,-1/len(text)*sum(result))

In [8]:
# Computes per-char perplexity of a text, given an input LM.  Smoothing is very, very simple, just using a small constant
def smoothed_perplexity(text, lm, order=4):
    # Pad the input with "~" chars.  This handles the case where order > len(text).
    pad = "~" * order
    data = pad + text
    result=[]
    flag =False
    for i in range(len(data)-order):
        if data[i:i+order] not in lm:
            flag = True
            result.append(log2(1.0e-7))
        else:
            subsetoflm =lm[data[i:i+order]]
        for key, value in subsetoflm:
            if key ==data[i+order]:
                result.append(log2(value))
    return  pow(2,-1/len(text)*sum(result)) 
    

# end of file

### (a) Simple Character LM. Examine the Python code provided (charlm.py). Then train an order 4 (i.e., 5-gram) model from the provided file subtitles.txt. 
## mylm = train_char_lm('subtitles.txt', 4)
### What are the continuations of 'atio' ? What about 'nivi' ? And 'supe' ?
### print_probs(mylm, 'atio')
### The generate_text method can produce random strings from a trained model. Generate some sample strings (of up to
### length 80 characters) from the model. Show us three of your favorite examples produced by the model.
### o 3 points: Appropriate continuations for atio, nivi, and supe.
### o 1 point: Random sentences 

In [9]:
# troubleshooting, not needed
lmT=train_char_lm("testTest.txt", 4)

In [10]:
lmTrained=train_char_lm("subtitles.txt", 4)

In [11]:
print_probs(lmTrained, "atio")

[('n', 0.9940436161014506),
 (' ', 0.00220962628494572),
 ('.', 0.0013930252665962147),
 (',', 0.0009607070804111826),
 ('?', 0.0003362474781439139),
 ("'", 0.00024017677010279565),
 ('u', 0.00019214141608223654),
 ('"', 0.0001441060620616774),
 ('s', 0.0001441060620616774),
 ('-', 9.607070804111827e-05),
 ('!', 4.8035354020559135e-05),
 (':', 4.8035354020559135e-05),
 ('m', 4.8035354020559135e-05),
 ('p', 4.8035354020559135e-05),
 ('r', 4.8035354020559135e-05)]


In [12]:
print_probs(lmTrained, "nivi")

[('n', 0.8), ('e', 0.1), ('s', 0.1)]


In [13]:
print_probs(lmTrained, "supe")

[('r', 0.9992144540455616), ('s', 0.0007855459544383347)]


In [14]:
generate_text(lmTrained, 4, nletters=200)

'See?\nWhat.\nHow about that lazy.\n- Diana German.\n"There\'s like you think I\'m immeasure you want?\nLook, hey.\nNo.\nI know what come?\nYou don\'t won with us.\n- We\'ll somebody dropped.\nGet that is well woman'

In [15]:
generate_text(lmTrained, 4, nletters=200)

'All redness sense.\nA Muslim could cup on the decision positive found the enough Praise shoot my shark?\n–Sustand I had no idea, but sleep, but of your oath.\nHell, even the appoint-night?\nWe shut up. We'

In [16]:
generate_text(lmTrained, 4, nletters=200)

'- Wow, eh?\nI\'m afraid."\nI\'LL FINALLY?\nOrson a prayer asshole later.\nPrevieve means she\'s nothing you say.\nYou\'re are protocolater it was ride that?\nHey!\nHe never.\nSpills have rebekah one else throw ma'

In [17]:
generate_text(lmTrained, 4, nletters=200)

'You are night talking a staff my last this far.\nNow I reasonalized him ther of Mexicans are best of Calified anything surve arent, Maria striots! Wake it we has been make his is the breason rathere.\n-'

### (b) Calculate perplexity. Extend the code from charlm.py to calculate a perplexity score for a provided string against a particular model of a specified order.1 Return positive infinity if any zero probabilities are encountered. 
### Hint 1: youwill want to work in log space. Hint 2: we advise reading the assigned chapter from the textbook and reviewing the lecture materials before implementing your perplexity function.
### perplexity('The boy loves his mother', mylm, 4)
### > 3.9092 (your result should be similar)
### To test your implementation, provide perplexity values for the following strings using a 5-gram model from subtitles.txt
### • The student loves homework
### • The yob loves homework
### • It is raining in London
### • asdfjkl; qwerty
### 2 points: Provides requested test cases
### 3 points: Handles zero probabilities
### 7 points: Correct implementation

In [18]:
 perplexity('The boy loves his mother', lmTrained, order=4)

~~~~The boy loves his mother


3.9091903673746224

In [19]:
perplexity('The student loves homework', lmTrained, order=4)

~~~~The student loves homework


4.606972940490917

In [20]:
perplexity('The yob loves homework', lmTrained, order=4)

~~~~The yob loves homework


inf

In [21]:
 perplexity('It is raining in London', lmTrained, order=4)

~~~~It is raining in London


3.7112360009044507

In [22]:
 perplexity('asdfjkl; qwerty', lmTrained, order=4)

~~~~asdfjkl; qwerty


inf

## (c) Naive smoothing. Further extend the code to perform some trivial smoothing for the perplexity function you implemented earlier. Instead of returning positive infinity for zero probabilities, implement a smoothed_perplexity method that uses 1.0e-7 for "zeros". As a consequence the sum of all probabilities in your model no longer add up to 1, but the effect will not be very dramatic. Test your method on the same strings from section (b).

### 8 points: Working smoothed perplexity and provided requested test cases

In [23]:
smoothed_perplexity('The boy loves his mother', lmTrained, order=4)

3.9091903673746224

In [24]:
smoothed_perplexity('The student loves homework', lmTrained, order=4)

4.606972940490917

In [25]:
smoothed_perplexity('The yob loves homework', lmTrained, order=4)

7.992378247505562

In [26]:
smoothed_perplexity('It is raining in London', lmTrained, order=4)

3.7112360009044507

In [27]:
smoothed_perplexity('asdfjkl; qwerty', lmTrained, order=4)

407954.987907739

### d) Language Identification. Training data for six European languages is provided. Create unigram (history=0) models for all six languages. Then make predictions for the provided test file which contains 1,200 lines. Note: each line in test.txt has two fields separated by a tab character; the first field is the correct language code:
### For each input string calculate the smoothed perplexity for the six separate models and return the language code for themodel with lowest smoothed_perplexity. For the first line of the test file (only), show perplexity scores for all sixlanguages. Calculate and report accuracy for the six languages - each number correct should be between 0 and 200 (e.g.,"de: 180 correct out of 200 lines - 90.0%"). Finally, repeat the experiment with higher-order n-grams. Try both a bigramand 4-gram model. How do your results change? You'll need to write code to load data, train models, calculate and sortscores, choose the lowest perplexity, and score results for the six languages.
### 1 point: Provide scores (perplexities) for each language on first test sentence.
### 2 points: Provides predictions for unigram model
### 2 points: Predictions for higher order models
### 3 points: Quality of results

In [28]:
def Language_Identification (order, firstSentence):
    da=train_char_lm("da.train.txt", order)
    de=train_char_lm("de.train.txt", order)
    en=train_char_lm("en.train.txt", order)
    fr=train_char_lm("fr.train.txt", order)
    it=train_char_lm("it.train.txt", order)
    nl=train_char_lm("nl.train.txt", order)
    langList=[da,de,en,fr,it,nl]
    langList2=['da','de','en','fr','it','nl']
    solution=[] # returned solution of full test set
    solution1=[] # returned solution of one sentence
    data= open('test.txt',encoding='utf-8').read()
#splitting data up
    answerQuestion = data.split('\n')    
    for i in range(len(answerQuestion)):
        answerQuestion[i]=answerQuestion[i].split('\t') 
# one sentence solution
    if firstSentence is True:
        resultsOfone={}
        for l in range(len(langList)):
#             print(langList2[l],smoothed_perplexity(answerQuestion[1][1], langList[l], order))
            resultsOfone.update({langList2[l]:smoothed_perplexity(answerQuestion[0][1], langList[l], order)})
        solution1.append({key:value for key, value in resultsOfone.items()}) 
        return solution1
#full solution 
    else:
        for i in range(len(answerQuestion)-1):
    #         print(answerQuestion[i][1]+'\n')  
            resultsOfTests={}
            for l in range(len(langList)):
#                 print(langList2[l],smoothed_perplexity(answerQuestion[i][1], langList[l], order))
#dict of 5 languages against one sentence
                resultsOfTests.update({langList2[l]:smoothed_perplexity(answerQuestion[i][1], langList[l], order)})
# list of best performing language per sentnece
            solution.append({key:value for key, value in resultsOfTests.items() if value ==min(resultsOfTests.values())}) 
    #         print(solution)
        return solution

In [29]:
#Score for each language for first sentence
Language_Identification(0,True)

[{'da': 28.990266275714863,
  'de': 29.17749118527597,
  'en': 20.393278329762353,
  'fr': 21.23036140889083,
  'it': 23.15318659107709,
  'nl': 26.3193826897344}]

In [30]:
#Score for each language for entire test set
x= Language_Identification(0,False)

In [31]:
#Score for each language for entire test set
x1= Language_Identification(4,False)

In [32]:
def truth_accuracy(order):
    x = Language_Identification(order,False)
    data= open('test.txt',encoding='utf-8').read()
#splitting data up
    answerQuestion = data.split('\n')    
    for i in range(len(answerQuestion)):
        answerQuestion[i]=answerQuestion[i].split('\t')
    cntda=0
    cntde=0
    cntfr=0
    cnten=0
    cntit=0
    cntnl=0
#     print(len(answerQuestion))
#     print(len(x))
    for i in range(len(answerQuestion)-1):
#         print(list(x[i].keys())[0],answerQuestion[i][0] )
        if list(x[i].keys())[0] ==answerQuestion[i][0]:
            if answerQuestion[i][0] == 'da':
                cntda+=1
            elif answerQuestion[i][0] == 'de':
                cntde+=1
            elif answerQuestion[i][0] == 'fr':
                cntfr+=1
            elif answerQuestion[i][0] == 'en':
                cnten+=1
            elif answerQuestion[i][0] == 'it':
                cntit+=1
            elif answerQuestion[i][0] == 'nl':
                cntnl+=1
    return {'da':cntda/200, 'de':cntde/200, 'fr':cntfr/200, 'en':cnten/200, 'it':cntit/200, 'nl':cntnl/200}
    

    


In [33]:
#resulting accuracy of 0 order
truth_accuracy(0)

{'da': 0.185, 'de': 0.505, 'fr': 0.205, 'en': 0.915, 'it': 0.8, 'nl': 0.855}

In [34]:
#resulting accuracy of 4 order
truth_accuracy(4)

{'da': 1.0, 'de': 0.995, 'fr': 0.995, 'en': 0.995, 'it': 0.995, 'nl': 0.995}

### (e) Gender Bias. For a different classification problem examine this data that contains questions that were asked to male or female tennis professionals after a televised match. Can you predict whether a question is addressed to a male or to a female player? The files are: tennis.{train,test}.txt and there are two fields separated by a tab. Like the language ID task, see how well you can classify male-directed vs. female-directed questions. See which length model works best. What is your best accuracy on the test set?
### 2 points: Calculate accuracies for both classes
### 3 points: Show results for models of at least three n-gram orders.
### 3 points: Quality of results.

In [35]:
def train_char_lmSports(sents, order):
#     data = open(fname,encoding='utf-8').read()
    lm = defaultdict(Counter)
    for s in sents:
        pad = "~" * order
        data = pad + s + '\n'
#         print(data)
        for i in range(len(data)-order):
            history, char = data[i:i+order], data[i+order]
#             print(history, char)
            lm[history][char]+=1
#             print(lm)
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

In [36]:
import re
def Gender_Bias(order):
    data= open('tennis.train.txt',encoding='utf-8').read()
    resultM = re.findall(r"^M\t(.*\n)",data, re.MULTILINE)      
    resultF = re.findall(r"^F\t(.*\n)",data, re.MULTILINE)
#     print(resultM[1])
    M=train_char_lmSports(resultM, order)
    F=train_char_lmSports(resultF, order)
    
    langList=[M,F]
    langList2=['M','F']
    solution=[] # returned solution of full test set
#splitting data up
    answerQuestion = data.split('\n')    
    for i in range(len(answerQuestion)):
        answerQuestion[i]=answerQuestion[i].split('\t') 
#full solution 
    else:
        for i in range(len(answerQuestion)-1):
    #         print(answerQuestion[i][1]+'\n')  
            resultsOfTests={}
            for l in range(len(langList)):
#                 print(langList2[l],smoothed_perplexity(answerQuestion[i][1], langList[l], order))
#dict of 2 languages against one sentence
                resultsOfTests.update({langList2[l]:smoothed_perplexity(answerQuestion[i][1], langList[l], order)})
# list of best performing language per sentnece
            solution.append({key:value for key, value in resultsOfTests.items() if value ==min(resultsOfTests.values())}) 
    #         print(solution)
    
    cntM=0
    cntF=0
#     print(len(answerQuestion))
#     print(len(x))
    for i in range(len(answerQuestion)-1):
#         print(list(x[i].keys())[0],answerQuestion[i][0] )
        if list(solution[i].keys())[0] ==answerQuestion[i][0]:
            if answerQuestion[i][0] == 'M':
                cntM+=1
            elif answerQuestion[i][0] == 'F':
                cntF+=1
    return {'M':cntM/39455, 'F':cntF/34264}
#         return solution


In [37]:
Gender_Bias(2)

{'M': 0.5915853503991889, 'F': 0.7263308428671492}

In [38]:
Gender_Bias(3)

{'M': 0.6580661513116208, 'F': 0.716407891664721}

In [39]:
Gender_Bias(4)

{'M': 0.6959827651755164, 'F': 0.7406899369600747}

##### 